# LimberJack.jl Tutorial

## Installation 

Once you have installed ```Julia``` you can install ```LimberJack.jl``` by:
1. Clone the git repository
2. From the repository directory open ```Julia```
3. In the ```Julia``` command line run:
``` julia
    using Pkg
    Pkg.add(path=".")
```

## Usage

In [25]:
# The statistical inference frame-work we will use
using Turing
# Our code
using LimberJack
# Some data management libs.
using CSV
using NPZ
using FITSIO
# Some Lin. Alg.
using LinearAlgebra
# Allows us to use Python when needed
using PythonCall
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/jaime/.julia/environments/v1.7/.CondaPkg/env/lib/python3.11/site-packages/numpy/__init__.py'>

### Data Preparation

```python

import numpy as np
import sacc
import yaml

def get_type(name, mode="write"):
    if ('DESwl' in name) or ('KiDS1000' in name):
        return 'e'
    elif "PLAcv" in name:
        if mode=="write":
            return 'k'
        if mode=="read":
            return "0"
    else:
        return '0'

sacc_path = "FD/cls_FD_covG.fits"
yaml_path = "DESY1/gcgc"
nzs_path = None #"DESY1/binned_40_nzs/"
fname = "DESY1/gcgc"

s = sacc.Sacc().load_fits(sacc_path)
with open(yaml_path+".yml") as f:
    config = yaml.safe_load(f)

# Apply scale cuts

indices = []
for cl in config['order']:
    t1, t2 = cl['tracers']
    lmin, lmax = cl['ell_cuts']
    type1 = get_type(t1, mode="read")
    type2 = get_type(t2, mode="read")
    cl_name = 'cl_%s%s' % (type1, type2)
    if cl_name == 'cl_e0':
        cl_name = 'cl_0e'
    ind = s.indices(cl_name, (t1, t2),
                    ell__gt=lmin, ell__lt=lmax)
    indices += list(ind)
s.keep_indices(indices)

cls = []
ls = []
indices = []
pairs = []
for cl in config['order']:
    t1, t2 = cl['tracers']
    type1 = get_type(t1, mode="read")
    type2 = get_type(t2, mode="read")
    cl_name = 'cl_%s%s' % (type1, type2)
    if cl_name == 'cl_e0':
        cl_name = 'cl_0e'
    l, c_ell, ind = s.get_ell_cl(cl_name, t1, t2,
                                 return_cov=False,
                                 return_ind=True)
    indices += list(ind)
    cls += list(c_ell)
    ls.append(l)
    type1 = get_type(t1, mode="write")
    type2 = get_type(t2, mode="write")
    pairs.append([t1+'_'+type1, t2+'_'+type2])

tracers = np.unique(pairs).flatten()
cov = s.covariance.dense[list(indices)][:, list(indices)]
w, v = np.linalg.eigh(cov)
cov = np.dot(v, np.dot(np.diag(np.fabs(w)), v.T))
cov = np.tril(cov) + np.triu(cov.T, 1)
inv_cov = np.linalg.inv(cov)

lengths = np.array([len(l) for l in ls])
lengths = np.concatenate([[0], lengths])
idx  = np.cumsum(lengths)

pairs_ids = []
for pair in pairs:
    t1, t2 = pair
    id1 = list(tracers).index(t1)
    id2 = list(tracers).index(t2)
    ids = [id1+1, id2+1]
    pairs_ids.append(ids)

indices = np.array(indices)
pairs = np.array(pairs)
pairs_ids = np.array(pairs_ids)
cls = np.array(cls)
idx = np.array(idx)
cov = np.array(cov)
inv_cov = np.array(inv_cov)

dict_save = {'tracers': tracers, 'pairs': pairs,
             'pairs_ids': pairs_ids, 'cls': cls, 'idx': idx,
             'cov': cov, 'inv_cov': inv_cov}

np.savez(fname+"_meta.npz", **dict_save)

###########

dict_save = {}

for pair, l in zip(pairs, ls):
    t1, t2 = pair
    print(t1, t2, len(l))
    dict_save[f'ls_{t1}_{t2}'] = np.array(l)

for name, tracer in s.tracers.items():
    name = name+'_'+get_type(name, mode="write")
    if name in tracers:
        if nzs_path is None:
            z=np.array(tracer.z)
            dndz=np.array(tracer.nz)
            dict_save[f'nz_{name}'] = np.array([z, dndz])
        else:
            nzs = np.load(nzs_path+f'nz_{name}.npz')
            z = nzs["z"]
            dndz = nzs["dndz"]
            dict_save[f'nz_{name}'] = np.array([z, dndz])

np.savez(fname+"_files.npz", **dict_save)


```

The purpose of this notebook is show the user how to interface ```LimberJack.jl``` with ```Turing.jl``` to perform statistical inference.
The first hurdle is converting our data into a format that ```LimberJack.jl``` can parse. 
The main obstacle is the lack of a ```Julia``` implementation of ```sacc.py``` (```yaml.jl``` exists!).
In the future we might want to either call ```sacc.py``` from ```Julia``` or code something like ```sacc.jl``` to stream line the process. 
However, at the moment the process is as follows:
1. The user must have a ```.yml``` specifying the order and $l$'s of the tracers considered as well as the the corresponding ```sacc.py``` file in ```.fits``` format. 
2. The user must direct script "../data/make_data.py" (see above) to the corresponding files in lines 16 and 17 of the script. Note that if the user wishes to use $n(z)$'s different from the ones contained in the $\text{.fits}$ file, they must also write the path to these $n(z)$'s in line 18. Finally, the user must specify a name for the output files in line 19.
3. Running ```python3 make_data.py``` will then generate two files: 
    + ```fname_meta.npz```
        + ```tracers```: a list of all tracers 
        + ```pairs```: a list of all pairs of tracers for the $C_l$'s
        + ```cls```: the concatenated flat data vector
        + ```idx```: indecies of the data vector corresponding to the first $l$ of each $C_l$ such that ```cli = cls[idx[i]:idx[i+1]]```
        + ```cov```: covariance matrix of the data vector
        + ```inv_cov```: inverse of the former
    + ```fname_files.npz```
        + ```ls_tracer_tracer```: an array containing the $l$'s for the tracer pair
        + ```nz_tracer```: the redshift distribution of galaxies of a given tracer as ```[z, dndz]```. 

Note that we will have to use ```Python``` (through ```PythonCall```) to load the ```meta``` file since ```Julia``` dictionaries only allow structures like ```Dict(type_a => type_b, type_a => type_b, type_a => type_b ...)``` but we want something like ```Dict(type_a => type_b, type_a => type_c, type_a => type_d ...)``` (Not just ```str``` to ```Array``` but ```str``` to ```Array``` and to ```Array[Array]``` etc)

### Load Data

In [46]:
# Load the processed data
fol = "LSST"
data_set = "gcgc_Nzs_40"
meta = np.load(string("../data/", fol, "/", data_set, "_meta.npz"))
files = npzread(string("../data/", fol, "/", data_set, "_files.npz"))

# Unpack the data
tracers = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"]);
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov = pyconvert(Matrix{Float64}, meta["cov"])
# whitening the data for better sampling
errs = sqrt.(diag(cov))
fake_data = data_vector ./ errs
fake_cov = Hermitian(cov./ (errs * errs'));

# Load the custom nz's
nz_path = "../data/DESY1/binned_40_nzs/"
zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, "DESgc__0_0")
zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, "DESgc__1_0")
zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, "DESgc__2_0")
zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, "DESgc__3_0")

(Float32[0.06, 0.099999994, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38, 0.42000002  …  1.26, 1.3, 1.34, 1.38, 1.4200001, 1.46, 1.5, 1.54, 1.5799999, 1.62], [0.0009240673730447433, 0.005814852428634714, 0.0043443927624754715, 0.0041160596873804104, 0.009895392517209878, 0.003943404553941104, 0.005722829819267932, 0.0033400048064060497, 0.007211179232808158, 0.0031238118244838213  …  0.031763332972974484, 0.00911825258939116, 0.011404821613995498, 0.014876208760203333, 0.013381480231233972, 0.01378891718945615, 0.007705600228305088, 0.0036701150619442577, 0.0035936918557433503, 0.003960094429040499], [1.3836301678444849e-6 -1.3977640396582891e-8 … -1.395804086718494e-10 1.0317296311408265e-9; -1.3977640396582891e-8 2.103089987389617e-5 … -3.98300461516264e-9 -3.6854268751077994e-9; … ; -1.395804086718494e-10 -3.98300461516264e-9 … 3.774187479133305e-6 3.6353211990832475e-9; 1.0317296311408265e-9 -3.6854268751077994e-9 … 3.6353211990832475e-9 4.229474497801167e-6])

### Defining the model

Now it is time to create the ```Turin.jl``` model. This is done through a function using the ```@model``` decorator.
The important detail here is that the model can only be a function of the data (upon which we will condition the model to obtain constraints). The model function must accomplish the following:
1. Define the priors for the parameters. This is done using the syntax of ```Distributions.jl``` for statistical programming. Note that the nuisance parameters must follow a strict naming convention in terms of the tracer they belong to. Otherwise, they will not be recognized. Also in order to pass the nuisance parameters our machinary they must be written as a dictionary. 
2. Initiate an instance of ```cosmo = LimberJack.Cosmology``` which will generate the matter power spectrum for the sampled cosmological parameters upon initializing. 
3. Evaluate ```LimberJack.Theory(cosmo, tracers, pairs, idx, files; Nuisances=nuisances)``` which for a given instance of  ```LimberJack.Cosmology``` it will return the concatenated $C_l$'s vector for the considered tracers, tracers' pairs, their indeces in the data vector and their auxialiry files  ($l$'s and $n(z)$'s).

In [39]:
@model function model(data; #data dependence!!!
                      tracers=tracers,
                      pairs=pairs,
                      idx=idx,
                      cov=fake_cov, 
                      files=files) 
    # Define priors
    Ωm ~ Uniform(0.2, 0.6)
    s8 ~ Uniform(0.6, 0.9)
    Ωb ~ Uniform(0.03, 0.07)
    h ~ Uniform(0.55, 0.91)
    ns ~ Uniform(0.87, 1.07)
    
    # create LimberJack.jl Cosmology instance
    cosmology = Cosmology(Ωm, Ωb, h, ns, s8;
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    # Define priors for nuisance parameters
    n = length(nz_k0)
    DESgc__0_0_nz = zeros(cosmology.settings.cosmo_type, n)
    DESgc__1_0_nz = zeros(cosmology.settings.cosmo_type, n)
    DESgc__2_0_nz = zeros(cosmology.settings.cosmo_type, n)
    DESgc__3_0_nz = zeros(cosmology.settings.cosmo_type, n)
    for i in 1:n
        DESgc__0_0_nz[i] ~ TruncatedNormal(nz_k0[i], sqrt.(diag(cov_k0))[i], 0.0, 1) 
        DESgc__1_0_nz[i] ~ TruncatedNormal(nz_k1[i], sqrt.(diag(cov_k1))[i], 0.0, 1) 
        DESgc__2_0_nz[i] ~ TruncatedNormal(nz_k2[i], sqrt.(diag(cov_k2))[i], 0.0, 1) 
        DESgc__3_0_nz[i] ~ TruncatedNormal(nz_k3[i], sqrt.(diag(cov_k3))[i], 0.0, 1) 
    end

    DESgc__0_0_b ~ Uniform(0.8, 3.0) # = 1.21
    DESgc__1_0_b ~ Uniform(0.8, 3.0) # = 1.30
    DESgc__2_0_b ~ Uniform(0.8, 3.0) # = 1.48
    DESgc__3_0_b ~ Uniform(0.8, 3.0) # = 1.64

    # Nuisances are taken in as a dictionary
    nuisances = Dict("DESgc__0_0_nz" => DESgc__0_0_nz,
                     "DESgc__1_0_nz" => DESgc__1_0_nz,
                     "DESgc__2_0_nz" => DESgc__2_0_nz,
                     "DESgc__3_0_nz" => DESgc__3_0_nz,
        
                     "DESgc__0_0_b" => DESgc__0_0_b,
                     "DESgc__1_0_b" => DESgc__1_0_b,
                     "DESgc__2_0_b" => DESgc__2_0_b,
                     "DESgc__3_0_b" => DESgc__3_0_b)

    # Obtains theoretical prediction 
    theory = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    
    # Defien likelihood
    data ~ MvNormal(theory ./ errs, cov)
end;

Now it is time to condition the model on the data. This is simply done as:

In [40]:
stat_model = model(fake_data);

### Sampling

Finally, we have to find a way of sampling our model.
We will do so using ```NUTS```.
Thankfully ```Turing.jl``` takes care of most of it. 
We will have however to come up with a schedule to segment the sampling process. 
1. Define number of cycles and likelihood evaluations for segment
2. Define specifications for ```NUTS``` sampler. 
3. Define folder to save chains to.
4. Check if there are samples already in the folder.
    + If there are samples, restart sampling process from last chain.
    + If not, start sampling from scratch.
5. Use ```CSV.jl``` to save the progress.
The key method is ```Turing.sample(stat_model, sampler, iterations; progress=true, save_state=true, resume_from=old_chain)``` samples the ```stat_model``` given a ```sampler``` for a number of ```iterations```. 
When restarting chains, we have to: 
1. Load the past chain ```old_chain = read(joinpath(folname, string("chain_", i-1,".jls")), Chains)```
2. Add the following flag to ```sample(...; resume_from=old_chain)```.

In [41]:
cycles = 6 # Segments
iterations = 100 # Length of segments
# NUTS specs.
TAP = 0.60 
adaptation = 100
init_ϵ = 0.05
sampler = NUTS(adaptation, TAP)

#Some prints
println("sampling settings: ")
println("cycles ", cycles)
println("iterations ", iterations)
println("TAP ", TAP)
println("adaptation ", adaptation)
println("init_ϵ ", init_ϵ)

# Folder for chains
folpath = "../chains"
folname = string("test_", "TAP_", TAP)
folname = joinpath(folpath, folname)

# Check if we are restarting a chain
if isdir(folname)
    fol_files = readdir(folname)
    println("Found existing file")
    if length(fol_files) != 0
        last_chain = last([file for file in fol_files if occursin("chain", file)])
        last_n = parse(Int, last_chain[7])
        println("Restarting chain")
    else
        last_n = 0
    end
else
    mkdir(folname)
    println(string("Created new folder ", folname))
    last_n = 0
end

# Start sampling
for i in (1+last_n):(last_n+cycles)
    if i == 1
        chain = sample(stat_model, sampler, iterations;
                       progress=true, save_state=true)
    else
        old_chain = read(joinpath(folname, string("chain_", i-1,".jls")), Chains)
        chain = sample(stat_model, sampler, iterations;
                       progress=true, save_state=true, resume_from=old_chain)
    end  
    # Save chains
    write(joinpath(folname, string("chain_", i,".jls")), chain)
    CSV.write(joinpath(folname, string("chain_", i,".csv")), chain)
    CSV.write(joinpath(folname, string("summary_", i,".csv")), describe(chain)[1])
end

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false

sampling settings: 
cycles 6
iterations 100
TAP 0.6
adaptation 100
init_ϵ 0.05
Found existing file


LoadError: InterruptException:

## Parallelization

Parallelization in ```Julia``` is a little bit obscure to me but this is how I do it. 
First we have to modify our script as follows:

In [ ]:
using Distributed

@everywhere using Turing
@everywhere using LimberJack
@everywhere using CSV
@everywhere using NPZ
@everywhere using FITSIO
@everywhere using LinearAlgebra
@everywhere using PythonCall
@everywhere np = pyimport("numpy")

@everywhere println("My id is ", myid(), " and I have ", Threads.nthreads(), " threads")

@everywhere fol = "LSST"
@everywhere data_set = "gcgc_Nzs_40"
@everywhere meta = np.load(string("../data/", fol, "/", data_set, "_meta.npz"))
@everywhere files = npzread(string("../data/", fol, "/", data_set, "_files.npz"))

@everywhere tracers_names = pyconvert(Vector{String}, meta["tracers"])
@everywhere pairs = pyconvert(Vector{Vector{String}}, meta["pairs"]);
@everywhere idx = pyconvert(Vector{Int}, meta["idx"])
@everywhere data_vector = pyconvert(Vector{Float64}, meta["cls"])
@everywhere cov_tot = pyconvert(Matrix{Float64}, meta["cov"])
@everywhere errs = sqrt.(diag(cov_tot))
@everywhere fake_data = data_vector ./ errs
@everywhere fake_cov = Hermitian(cov_tot ./ (errs * errs'));

@everywhere nz_path = "../data/DESY1/binned_40_nzs/"
@everywhere zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, "DESgc__0_0")
@everywhere zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, "DESgc__1_0")
@everywhere zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, "DESgc__2_0")
@everywhere zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, "DESgc__3_0")

@everywhere @model function model(data;
                                  tracers_names=tracers_names,
                                  pairs=pairs,
                                  idx=idx,
                                  cov=fake_cov, 
                                  files=files) 
    Ωm ~ Uniform(0.2, 0.6)
    s8 = 0.811 #~ Uniform(0.6, 0.9)
    Ωb ~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.55, 0.91)
    ns ~ Uniform(0.87, 1.07)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8;
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    n = length(nz_k0)
    DESgc__0_0_nz = zeros(cosmology.settings.cosmo_type, n)
    DESgc__1_0_nz = zeros(cosmology.settings.cosmo_type, n)
    DESgc__2_0_nz = zeros(cosmology.settings.cosmo_type, n)
    DESgc__3_0_nz = zeros(cosmology.settings.cosmo_type, n)
    for i in 1:n
        DESgc__0_0_nz[i] ~ TruncatedNormal(nz_k0[i], sqrt.(diag(cov_k0))[i], 0.0, 1) 
        DESgc__1_0_nz[i] ~ TruncatedNormal(nz_k1[i], sqrt.(diag(cov_k1))[i], 0.0, 1) 
        DESgc__2_0_nz[i] ~ TruncatedNormal(nz_k2[i], sqrt.(diag(cov_k2))[i], 0.0, 1) 
        DESgc__3_0_nz[i] ~ TruncatedNormal(nz_k3[i], sqrt.(diag(cov_k3))[i], 0.0, 1) 
    end

    DESgc__0_0_b ~ Uniform(0.8, 3.0) # = 1.21
    DESgc__1_0_b ~ Uniform(0.8, 3.0) # = 1.30
    DESgc__2_0_b ~ Uniform(0.8, 3.0) # = 1.48
    DESgc__3_0_b ~ Uniform(0.8, 3.0) # = 1.64


    nuisances = Dict("DESgc__0_0_nz" => DESgc__0_0_nz,
                     "DESgc__1_0_nz" => DESgc__1_0_nz,
                     "DESgc__2_0_nz" => DESgc__2_0_nz,
                     "DESgc__3_0_nz" => DESgc__3_0_nz,
        
                     "DESgc__0_0_b" => DESgc__0_0_b,
                     "DESgc__1_0_b" => DESgc__1_0_b,
                     "DESgc__2_0_b" => DESgc__2_0_b,
                     "DESgc__3_0_b" => DESgc__3_0_b)
    
    theory = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    data ~ MvNormal(theory ./ errs, cov)
end;

cycles = 6 # Segments
iterations = 100 # Length of segments
# NUTS specs.
TAP = 0.60 
adaptation = 100
init_ϵ = 0.05

println("sampling settings: ")
println("cycles ", cycles)
println("iterations ", iterations)
println("TAP ", TAP)
println("adaptation ", adaptation)
println("init_ϵ ", init_ϵ)

folpath = "../chains"
folname = string("test_", "TAP_", TAP)
folname = joinpath(folpath, folname)

if isdir(folname)
    fol_files = readdir(folname)
    println("Found existing file")
    if length(fol_files) != 0
        last_chain = last([file for file in fol_files if occursin("chain", file)])
        last_n = parse(Int, last_chain[7])
        println("Restarting chain")
    else
        last_n = 0
    end
else
    mkdir(folname)
    println(string("Created new folder ", folname))
    last_n = 0
end

for i in (1+last_n):(last_n+cycles)
    if i == 1
        chain = sample(model(fake_data), NUTS(adaptation, TAP), MCMCDistributed(), iterations, nchains;
                       progress=true, save_state=true)
    else
        old_chain = read(joinpath(folname, string("chain_", i-1,".jls")), Chains)
        chain = sample(model(fake_data), NUTS(adaptation, TAP), MCMCDistributed(), iterations, nchains;
                       progress=true, save_state=true, resume_from=old_chain)
    end  
    write(joinpath(folname, string("chain_", i,".jls")), chain)
    CSV.write(joinpath(folname, string("chain_", i,".csv")), chain)
    CSV.write(joinpath(folname, string("summary_", i,".csv")), describe(chain)[1])
end

Here ```Distributed``` is the library managing the parallelization.
Then, each line that is evaluated by all workers must have the decorator ```@everywhere```.
Finally, in order to parallelize ```Turing``` we have to add the following inputs to ```sample(stat_model, sampler, MCMCDistributed(), iterations, nchains; progress=true, save_state=true)```

## Glamdring

To run the previous script I use:
```addqueue -e -n 4x4 -m 10 -s -c "blabla" -q cmb /usr/local/shared/julia-1.7.1/bin/julia -p 4 -t 4 script.jl```.

This will run 4 chains each using 4 threads in parallel

## Plotting

In order to plot the results I use ```GetDist.py``` as follows:

```python
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import os
import getdist
from getdist import plots, MCSamples
import sacc

def add_chains(path):
    chains = []
    i = 1 
    while os.path.isfile(path+"chain_{}.csv".format(i)):
        chain = pd.read_csv(path+"chain_{}.csv".format(i))
        chains.append(chain)
        i += 1
    return pd.concat(chains)

test = add_chains("../chains/test_TAP_0.65/")

labels_dict = {'eta': '\eta',
               'l': 'l',
               'h': 'h',
               'Ωm': '\Omega_m',
               'Ωb': '\Omega_b',
               'ns': 'n_s',
               's8': '\sigma_8',
               'S8': 'S_8',
               'A_IA': 'A_{IA}',
               'alpha_IA': r'\alpha_{IA}',
               'DESgc__0_0_dz': 'dz_{DESY1gc \, 0}',
               
               'DESgc__1_0_dz': 'dz_{DESY1gc \, 1}',
               'DESgc__2_0_dz': 'dz_{DESY1gc \, 2}',
               'DESgc__3_0_dz': 'dz_{DESY1gc \, 3}',
               'DESgc__4_0_dz': 'dz_{DESY1gc \, 4}',
               
               'DESwl__0_e_dz': 'dz_{DESY1wl \, 0}',
               'DESwl__1_e_dz': 'dz_{DESY1wl \, 1}',
               'DESwl__2_e_dz': 'dz_{DESY1wl \, 2}',
               'DESwl__3_e_dz': 'dz_{DESY1wl \, 3}',
               
               'DESgc__0_0_b': 'b_{DESY1 \, 0}',
               'DESgc__1_0_b': 'b_{DESY1 \, 1}',
               'DESgc__2_0_b': 'b_{DESY1 \, 2}',
               'DESgc__3_0_b': 'b_{DESY1 \, 3}',
               'DESgc__4_0_b': 'b_{DESY1 \, 4}',
               
               'DESwl__0_e_m': 'm_{DESY1 \, 0 }',
               'DESwl__1_e_m': 'm_{DESY1 \, 1 }',
               'DESwl__2_e_m': 'm_{DESY1 \, 2 }', 
               'DESwl__3_e_m': 'm_{DESY1 \, 3 }',
               
               'eBOSS__0_0_b': 'b_{eBOSS \, 0}',
               'eBOSS__1_0_b': 'b_{eBOSS \, 1}',
               
               "DECALS__0_0_b": 'b_{DECALS \, 0}',
               "DECALS__1_0_b": 'b_{DECALS \, 1}',
               "DECALS__2_0_b": 'b_{DECALS \, 2}',
               "DECALS__3_0_b": 'b_{DECALS \, 3}',
               
               "DECALS__0_0_dz": 'dz_{DECALS \, 0}',
               "DECALS__1_0_dz": 'dz_{DECALS \, 1}',
               "DECALS__2_0_dz": 'dz_{DECALS \, 2}',
               "DECALS__3_0_dz": 'dz_{DECALS \, 3}',

               "KiDS1000__0_e_dz": 'dz_{KiDS \, 0}',
               "KiDS1000__1_e_dz": 'dz_{KiDS \, 1}',
               "KiDS1000__2_e_dz": 'dz_{KiDS \, 2}',
               "KiDS1000__3_e_dz": 'dz_{KiDS \, 3}',
               "KiDS1000__4_e_dz": 'dz_{KiDS \, 4}',
               
               "KiDS1000__0_e_m": 'm_{KiDS \, 0}',
               "KiDS1000__1_e_m": 'm_{KiDS \, 1}',
               "KiDS1000__2_e_m": 'm_{KiDS \, 2}',
               "KiDS1000__3_e_m": 'm_{KiDS \, 3}',
               "KiDS1000__4_e_m": 'm_{KiDS \, 4}',
               
               "v[1]": "v_{1}", "v[2]": "v_{2}",
               "v[3]": "v_{3}", "v[4]": "v_{4}",
               "v[5]": "v_{5}", "v[6]": "v_{6}",
               "v[7]": "v_{7}", "v[8]": "v_{8}", 
               "v[9]": "v_{9}", "v[10]": "v_{10}",
               "v[11]": "v_{11}"}

def make_chain(file, label, ranges=dict({})):
    params = np.array(list(file.keys()))
    names = []
    labels = []
    samples = []
    print()
    print(label)
    for param in params:
        print(param)
        if param in labels_dict.keys():
            print(param)
            names.append(param) 
            labels.append(labels_dict[param]) 
            samples.append(file[param])
    if ('s8' in params) & ('Ωm' in params):
        print('S8')
        names.append('S8')
        labels.append(labels_dict['S8'])
        samples.append(file['s8']*np.sqrt(file['Ωm']/0.3))

    names = np.array(names)
    labels = np.array(labels)
    samples = np.transpose(np.array(samples))
    print("========")

    return MCSamples(samples=samples, names=names, labels=labels, label=label, ranges=ranges,  
                    settings={'mult_bias_correction_order':0,'smooth_scale_2D':0.4, 'smooth_scale_1D':0.3})

test_samples = make_chain(test, "Test");

g = plots.getSubplotPlotter(subplot_size=4)
g.triangle_plot(test_samples, filled=True)
```